In [60]:
import numpy as np
import pandas as pd
from pandas_datareader import data
import matplotlib.pyplot as plt


In [61]:
#get data
df = data.DataReader(['PCY', 'IECIX','TRECX','0P0000KSPA.L','0P0001HIXD.L','UIQI.DE','DAPP','ERTH','QGMIX','SCD.JO'], 'yahoo', start='2021/04/16', end='2022/09/03')
df = df['Adj Close']
df = df.dropna()


#Covariance
cov_matrix = df.pct_change().apply(lambda x: np.log(1+x)).cov()

#correlation
corr_matrix = df.pct_change().apply(lambda x: np.log(1+x)).corr()


# resample to get yearly average expected return for individual assets , here use arithmetic mean
#ind_er = df.resample('Y').last().pct_change().mean()
ind_er = pd.Series(np.array([0.052,0.059,0.048,0.046,0.067,0.07,0.041,0.0525,0.0591,0.0993],float),index=['PCY', 'IECIX','TRECX','0P0000KSPA.L','0P0001HIXD.L','UIQI.DE','DAPP','ERTH','QGMIX','SCD.JO'])


#get annual std

ann_sd = df.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(250))

# get summarization table for individual annual expected return and expected std

assets = pd.concat([ind_er, ann_sd], axis=1) # Creating a table for visualising returns and volatility of assets
assets.columns = ['Returns', 'Volatility']

#getting the possibilities of portfolios with random function

p_ret = [] # Define an empty array for portfolio returns
p_vol = [] # Define an empty array for portfolio volatility
p_weights = [] # Define an empty array for asset weights

num_assets = len(df.columns)
num_portfolios = 50000 # number of simulation

for portfolio in range(num_portfolios):
    weights = np.random.random(num_assets) # return number between 0-1
    weights = weights/np.sum(weights)
    p_weights.append(weights)
    returns = np.dot(weights, ind_er) # Returns are the product of individual expected returns of asset and its 
                                      # weights 
    p_ret.append(returns)
    var = cov_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum()# Portfolio Variance
    sd = np.sqrt(var) # Daily standard deviation
    ann_sd = sd*np.sqrt(250) # Annual standard deviation = volatility
    p_vol.append(ann_sd)
    
data = {'Returns':p_ret, 'Volatility':p_vol}

for counter, symbol in enumerate(df.columns.tolist()):
    #print(counter, symbol)
    data[symbol+' weight'] = [w[counter] for w in p_weights] 

portfolios  = pd.DataFrame(data)


# get efficiency frontier


# get the combination with maximum sharpe ratio

rf = 0.0225
optimal_risky_port = portfolios.iloc[((portfolios['Returns']-rf)/portfolios['Volatility']).idxmax()]
plt.subplots(figsize=(10, 10))
plt.scatter(portfolios['Volatility'], portfolios['Returns'],marker='o', s=10, alpha=0.3)
plt.scatter(optimal_risky_port[1], optimal_risky_port[0], color='g', marker='*', s=100)

print(optimal_risky_port)




0.15590713725981936
